# про декораторы

в объектах все :
* числа
* строки
* классы
* функции

### функции высших порядков - функции на вход принимают другие функции и возвращают на выходе функции

#### определять функции внутри других

In [1]:
def wrapper_f():
    def hello_world():
        print('test')
    hello_world()
wrapper_f()

def hello_world_1():
    print('test')
hello_world = hello_world_1

test


In [2]:
def higher_order(func):
    print(f'получена функция {func}')
    func()
    return func

higher_order(hello_world)

получена функция <function hello_world_1 at 0x0000023B7F3639D0>
test


<function __main__.hello_world_1()>

#### пример

In [3]:
def decorator_function(func):
    def wrapper():
        print('Функция-обёртка!')
        print('Оборачиваемая функция: {}'.format(func))
        print('Выполняем обёрнутую функцию...')
        func()
        print('Выходим из обёртки')
    return wrapper

In [4]:
@decorator_function
def hello_world():
    print('hello world')

In [5]:
hello_world()

Функция-обёртка!
Оборачиваемая функция: <function hello_world at 0x0000023B7F363430>
Выполняем обёрнутую функцию...
hello world
Выходим из обёртки


In [6]:
hello_world_1()

test


In [7]:
hello_world = decorator_function(hello_world_1)
hello_world()

Функция-обёртка!
Оборачиваемая функция: <function hello_world_1 at 0x0000023B7F3639D0>
Выполняем обёрнутую функцию...
test
Выходим из обёртки


### использование декоратора для обертки в бенчмарк запроса

In [8]:
%%timeit
def benchmark(func):
    import time
    
    def wrapper():
        start = time.time()
        func()
        end = time.time()
        print(f'[*] Время выполнения: {end-start} секунд.')
    return wrapper

@benchmark
def fetch_webpage():
    import requests
    webpage = requests.get('https://www.kenrockwell.com/')

fetch_webpage()

[*] Время выполнения: 0.6836259365081787 секунд.
[*] Время выполнения: 0.5436699390411377 секунд.
[*] Время выполнения: 0.5227141380310059 секунд.
[*] Время выполнения: 0.5183618068695068 секунд.
[*] Время выполнения: 0.5445256233215332 секунд.
[*] Время выполнения: 0.5444567203521729 секунд.
[*] Время выполнения: 0.5556299686431885 секунд.
[*] Время выполнения: 0.5183873176574707 секунд.
535 ms ± 14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## декоратор с переменными

In [9]:
def benchmark(iters):
    def actual_decorator(func):
        import time
        
        def wrapper(*args, **kwargs):
            total = 0
            for i in range(iters):
                start = time.time()
                return_value = func(*args, **kwargs)
                end = time.time()
                total = total + (end-start)
            print('[*] Среднее время выполнения: {} секунд.'.format(total/iters))
            return return_value

        return wrapper
    return actual_decorator


@benchmark(iters=10)
def fetch_webpage(url):
    import requests
    webpage = requests.get(url)
    return webpage.text

webpage = fetch_webpage('https://google.com')
# print(webpage)

[*] Среднее время выполнения: 0.19495186805725098 секунд.


In [10]:
a = [1, 2, 3]

def outputter(func):
    def wrapper(*args):
        return_value = func(*args)
        print(return_value, 'из-за обертки вывелось')
        return return_value
    return wrapper
    
@outputter
def mysum(list_ar):
    sum_ = 0
    for i in list_ar:
        sum_ += i
    return sum_

mysum(a)

6 из-за обертки вывелось


6

In [11]:
a = [1, '2', 3]

def outputter(func):
    def wrapper(bad_inp):
        listargs = [int(s) for s in bad_inp]
        return_value = func(listargs)
        print(f'Сумма = {return_value} сумма посчиталась от списка, который имел строки')
        return return_value
    return wrapper
    
@outputter
def mysum(list_ar):
    sum_ = 0
    for i in list_ar:
        sum_ += i
    return sum_

mysum(a)

Сумма = 6 сумма посчиталась от списка, который имел строки


6

# Алгоритмы

## наивная рекурсия

In [12]:
def naive_fi(month):
    if month == 0 or month == 1:
        return 1
    else:
        return naive_fi(month - 1) + naive_fi(month - 2)
naive_fi(12)

233

In [16]:
%%timeit
naive_fi(20)

4.01 ms ± 714 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**одни вещи пересчитываются по много раз**

In [17]:
def dynamic_alg(month):
    num_bun_month = [0]*(month +2)
    num_bun_month[1] = 1
    for i in range(2, month + 2):
        num_bun_month[i] = num_bun_month[i-1] + num_bun_month[i-2]
    return num_bun_month[-1]

dynamic_alg(20)

10946

In [18]:
%%timeit
dynamic_alg(20)

3.97 µs ± 29.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


#### в 1000 раз быстрее

## O(n) 'О большое' нотация